In [ ]:
#Hand gesture recognizer
!python -m pip install opencv-python
!python -m pip install tensorflow
!python -m pip install mediapipe

In [ ]:
import numpy as np

import cv2

import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model

In [ ]:
#Initializing Mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

In [ ]:
#Loading the gestures Recognition model
model = load_model("model_data_download")

#Load class names
f = open("gestures.names", "r")
classNames = f.read().split('\n')
f.close()
print(classNames)

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    
    x, y, c = frame.shape
    
    #Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(framergb)
    
    className = ''
    
    #post process the result 
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)
                
                landmarks.append([lmx, lmy])
                
                mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
                
                prediction = model.predict([landmarks])
                classID = np.argmax(prediction)
                className = className[classID]
                
    cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
               1, (0,0,255), 2, cv2.LINE_AA)
    
    cv2.imshow("Output", frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()